# In this Notebook...
- Reinforcement Learning
    - Environment
    - Replay Buffer
    - Neural Network
    - Agent
    - Hyperparameters
    - Training Loop

# Notes

In [ ]:
# Rewards and Punishments

#rewards:
#containers with same prio close to each other
#keep stacks as low as possible

#punishments:
#amount of moves it takes to get all containers of one prio class out of created lay-out
#container with lower prio on top of container with high prio

In [6]:
#Remaining checklist

#environment: game over when shiplist empty
#new score function: calculate score by counting moves it takes to remove containers from layout
#checkmiddle function: when 4 or more wide, fix that stack can't be placed in middle


# Imports

In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

using gpu 0


In [2]:
#Is GPU avaible? if not, use cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Functions

In [3]:
def generateEnvironment(size=[3,3,2]):
    c =  [[['0'] * size[2]] * size[1]] * size[0]
    cSize = size[0]*size[1]*size[2]*3
    actionSize = size[0]*size[1]*size[2]
    speelveld = np.array(c)
    return speelveld, size

In [4]:
generateEnvironment()

(array([[['0', '0'],
         ['0', '0'],
         ['0', '0']],
 
        [['0', '0'],
         ['0', '0'],
         ['0', '0']],
 
        [['0', '0'],
         ['0', '0'],
         ['0', '0']]], dtype='<U1'),
 [3, 3, 2])

In [20]:
#From X,Y,Z coördinates to index in array
def coord_to_ind(X, Y, Z, lot_size):
    grid = np.arange(np.prod(lot_size)).reshape(lot_size)
    ind = grid[X][Y][Z]
    return ind

#From index in array to X,Y,Z coördinates
def ind_to_coord(ind, lot_size):
    grid = np.arange(np.prod(lot_size)).reshape(lot_size)
    coord = np.where(grid[:,:,:] == ind)
    X = coord[0][0]
    Y = coord[1][0]
    Z = coord[2][0]
    return X, Y, Z

#test some code
c = ind_to_coord(0, [1,1,1])
e = generateEnvironment()
a = AddContainer(e[0],e[1],1,c[0],c[1],c[2])
print(c)

(0, 0, 0)


# Environment

In [9]:
class environ:
    
    def __init__(self, lot_size=generateEnvironment()[1]):
        if not isinstance(lot_size, (list, tuple)) or not len(lot_size) == 3:
            raise ValueError('UNACCEPTABLE LOT_SIZE >:(')
            
        self.lot_size = lot_size
        self.max_x = lot_size[0]-1
        self.max_y = lot_size[1]-1
        self.max_z = lot_size[2]-1
        self.speelveld = generateEnvironment()[0]
        self.speelveld_ind = np.prod(lot_size)
         
    
    def step(self, act):#'act' moet int zijn
        
        #reward = 0
        act_loc = ind_to_coord(act, self.lot_size)
        new_speelveld = AddContainer(self.speelveld, self.lot_size, random.randint(1,3), act_loc[0], act_loc[1], act_loc[2])
        
        #game_over when yard is full, no more containers can be added to yard or ship is empty
        gameover = True
        for a in range(self.max_x+1):
            if new_speelveld[a][0][self.max_z] == '0' or new_speelveld[a][self.max_y][self.max_z] == '0':
                gameover = False
                break
        
        
        if not gameover:
            #place container on avaible space
            if not (self.speelveld == new_speelveld).all():
                reward = 1
                self.speelveld = new_speelveld
            #place container on unavaible space
            else:
                reward = -1
                self.speelveld = new_speelveld
                
        #no container can be added to lot
        else:
            #containeryard is filled
            if not '0' in new_speelveld:
                reward = 3
                self.reset()
            #containeryard still had space
            else:
                reward = -3
                self.reset()
            
        return new_speelveld, reward, gameover #array, int, bool
    
    #reset yard to empty
    def reset(self):
        
        self.speelveld = generateEnvironment()[0]
        return self.speelveld
    #def render(self):
        

In [ ]:
    #Environment Restrictions (from Martti)
    #-------------------------------------------------------------
    #Checks if current position is not floating.
    def CheckBelow(Environment, X, Y, Z):
        return Z == 0 or (Environment[X,Y,:Z] != '0').all()

    #Checks if current position has container.
    def CheckPos(Environment, Size, X, Y, Z):
        if not ExceedsLot(Size, X, Y, Z):
            return Environment[X, Y, Z] != '0'
        else:
            return False

    #Checks if coordinates exceed lot.
    def ExceedsLot(Size, X, Y, Z):
        return Size[2] <= Z or Size[1] <= Y or Size[0] <= X or Z < 0 or Y < 0 or X < 0

    #FIX, when 4 or more wide, fix that stack can't be placed in middle
    #Check whether container is in the middle of other containers.
    def CheckMiddle(Environment, Size, X, Y, Z):
        return CheckPos(Environment, Size, X, Y - 1, 0) and CheckPos(Environment, Size, X, Y + 1, 0)

    #Adds container to environment using state.
    def AddContainer(Environment, Size, State, X, Y, Z):
        tempEnvironment = np.copy(Environment)
        if CheckBelow(Environment, X, Y, Z) and not CheckPos(Environment, Size, X, Y, Z) and not ExceedsLot(Size, X, Y, Z) and not CheckMiddle(Environment, Size, X, Y, Z):
            tempEnvironment[X,Y,Z] = State
        return tempEnvironment
    #--------------------------------------------------------------   

# Replay Buffer

In [7]:
class ReplayBuffer():
    def __init__(self, size_max=10000):
        self.storagelst = []
        self.size_max = size_max
        self.ptr = 0
    
    #Add data to storage
    def add(self, data):#data contains state, state_next, action, reward and done
        #when storage reach max, replace first in storage
        if len(self.storagelst) == self.size_max:
            self.storagelst[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.size_max
        #add to storage
        else:
            self.storagelst.append(data)
    
    #take random sample from storage list
    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storagelst), size=batch_size)
        st, stn, act, rew, don = [], [], [], [], []
        
        for i in ind: 
            S, N, A, R, D = self.storage[i]
            st.append(np.array(S, copy=False))
            stn.append(np.array(N, copy=False))
            act.append(np.array(A, copy=False))
            rew.append(np.array(R, copy=False))
            don.append(np.array(D, copy=False))
        
        return np.array(st), np.array(stn), np.array(act).reshape(-1,1), np.array(rew).reshape(-1,1), np.array(don).reshape(-1,1)

# Neural Network

In [12]:
#DQN
class NeurNet(nn.Module):
    def __init__(self, inputsize, outputsize, hiddensize):
        super(NeurNet, self).__init__()
        self.denselayer_1 = nn.Linear(inputsize, hiddensize)
        self.denselayer_2 = nn.Linear(hiddensize, hiddensize)
        self.output = nn.Linear(hiddensize, outputsize)
        
        #self.loss = nn.MSEloss()
        #self.optimizer = optim.Adam(self.parameters(), lr=lrate)
        
    def forward(self, x):
        x = F.relu(self.denselayer_1(x))
        x = F.relu(self.denselayer_2(x))
        return self.output(x)

# Agent

In [13]:
class Agent:
    def __init__(self, inputsize, outputsize, hiddensize, lot_space, lrate):
        self.actionspace = [i for i in range(lot_space)]
        self.outputsize = outputsize
        self.trainnet = NeurNet(inputsize, outputsize, hiddensize).to(device)
        self.targnet = NeurNet(inputsize, outputsize, hiddensize).to(device)
        self.targnet.load_state_dict(self.trainnet.state_dict())
        self.optimizer = optim.Adam(self.trainnet.parameters(), lr=lrate)
        
    def action(self, state, epsilon):
        #if epsilon still high, higher chance of random
        if np.random.rand() <= epsilon:
            act = random.choice(self.actionspace)
        
        #put input into neural network and get an action
        else:
            with torch.no_grad():
                inputstate = torch.FloatTensor(state).to(device)
                act = self.trainnet(inputstate).max(0)[1].view(-1)
                act = int(action)
        return act #index of place in array where container will be put (int)
    
    
    def train(self, replay_buffer, batch_size, disc_gamma):
        #take sample from replay buffer storagelist
        st_nu, st_next, act, rew, done = replay_buffer.sample(batch_size)
        
        #batches to tensors
        state_batch = torch.FloatTensor(st_nu).to(device)
        next_state_batch = torch.FloatTensor(st_next).to(device)
        action_batch = torch.LongTensor(act).to(device)
        reward_batch = torch.FloatTensor(rew).to(device)
        done_batch = torch.FloatTensor(1 - done).to(device)
        
        #get train and target net Q values
        train_q = self.trainnet(state_batch).gather(1, action_batch)
        
        with torch.no_grad():
            targnet_q = reward_batch + done_batch * disc_gamma * \
                     torch.max( self.targnet(next_state_batch).detach(), dim=1)[0].view(batch_size, -1)
        
        #loss function
        loss_func = nn.MSELoss()
        loss = loss_func(train_q, targnet_q)
        #optimize parameters
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.detach().cpu().numpy()
    
    #update target network
    def update_netw(self, num_iter, update_every):
        #only update sometimes
        if num_iter % update_every == 0:
            self.targnet.load_state_dict(self.trainnet.state_dict())
        

# Hyperparameters

In [14]:
#hyperparameters
hidden_size = 64 #hiddenlayer size of neurnet
episodes = 1000 #amount of games
start_train_after = 100 #after this many steps start training
learning_rate = 0.0005
update_target_by = 1000 #update target network after this many steps
batch_size = 64 # batch size used for train network
discount_gamma = 0.9

#epsilon
epsilon_start = 1.0
epsilon_min = 0.001
epsilon_decay_by = episodes * 0.15
epsilon_decrease = (epsilon_start - epsilon_min)/(epsilon_decay_by)

#make environment
lot_size = generateEnvironment()[1]
env = environ(lot_size)
inputsize = lot_size[0] * lot_size[1] * lot_size[2]
outputsize = lot_size[0] * lot_size[1] * lot_size[2]
lotspace = lot_size[0] * lot_size[1] * lot_size[2]

#make replay buffer
replay_size = 10000
replay_buffer = ReplayBuffer(max_size=replay_size)

#make agent
agent = Agent(inputsize,outputsize,hidden_size,lotspace,learning_rate)

# Loop Training

In [125]:
list_stat_rewards = []
stats_by = 10
total_reward = 0
timesteps = 0
episode_len = 0
epsilon = epsilon_start

for ep in range(episodes):
    state = env.reset()
    state_loss = 0
    
    #train each episode
    while True:
        timesteps += 1
        #select action
        action = agent.action(state, epsilon)
        #apply action to environment
        next_state, reward, done = env.step(action)
        total_reward += reward
        episode_len += 1
        
        #when agent is done
        if done:
            next_state = np.zeros(state.shape)
            reward = -1
            #save experience to replay buffer
            replay_buffer.add((state, next_state, action, reward, done))
            
            list_stat_rewards.append((ep, total_reward, episode_len))
            total_reward = 0
            episode_len = 0
            
            epsilon -= epsilon_decrease
            if epsilon < epsilon_min:
                epsilon = epsilon_min
                
            if timesteps > start_train_after and ep % stats_by == 0:
                print('Episode: {}'.format(ep),
                    'Timestep: {}'.format(timesteps),
                    'Total reward: {:.1f}'.format(np.mean(list_stat_rewards[-stats_every:],axis=0)[1]),
                    'Episode length: {:.1f}'.format(np.mean(list_stat_rewards[-stats_every:],axis=0)[2]),
                    'Epsilon: {:.2f}'.format(epsilon),
                    'Loss: {:.3f}'.format(stats_loss))
            break
        
        #in both cases, save experience to replay buffer
        else:
            replay_buffer.add((state, next_state, action, reward, done))
            
        state = next_state
        
        #if enough steps have been completed to start training
        if timesteps > start_train_after:
            
            #train agent
            stats_loss += agent.train(replay_buffer, batch_size, discount_gamma)
            #update network
            agent.update_netw(timesteps, update_target_by)

/tmp/ipykernel_9561/4183677267.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x), np.array(y), np.array(u).reshape(-1,1), np.array(r).reshape(-1,1), np.array(d).reshape(-1,1)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.